In [ ]:
!pip install opencv-python

In [ ]:
# run if you are in Colab environment
from google.colab import drive
drive.mount("/content/MyDrive")

import numpy as np
from PIL import Image

Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


In [ ]:
# change the paths accordingly
path = "/content/MyDrive/MyDrive/hell/"
# select the desired image
image = "ravi.jpg"

### Script 1

Μια RGB εικόνα την περνάει από Bayer φίλτρο

- στο image_rgb.txt είναι τα rgb της εικόνας περασμένα σε txt για σύγκριση
- στο image_8bit.txt η εικόνα έχει περάσει από το bayer φίλτρο, από τα RGB έχουμε κρατήσει μόνο αυτό που πρέπει και η τιμή έχει μετατραπεί σε binary


In [ ]:
input_image = Image.open(path + image)
pixels = input_image.load()
rows, columns = input_image.size

print("rows: ", rows)
print("columns: ", columns)

if rows == 256 and columns == 256:
  # print("Image needs to resize to: ", 256)

  # input_image = input_image.resize((256, 256))
  # input_image.save(path + image)

  # pixels = input_image.load()

  image_rgb_file = open(path + "image_rgb.txt", 'w')
  image_8bit_file = open(path + "image_8bit.txt", 'w')

  for y in range(columns):
    for x in range(rows):
      rgb = pixels[x, y]
      rgb_to_string = str(rgb)
      rgb_to_string = rgb_to_string.replace(" ", "")
      rgb_to_string = rgb_to_string + " "
      image_rgb_file.write(rgb_to_string)

      r = rgb[0]
      g = rgb[1]
      b = rgb[2]

      r_to_bin = format(r, '08b')
      g_to_bin = format(g, '08b')
      b_to_bin = format(b, '08b')

      if (y % 2) == 0 and (x % 2) == 0:
        image_8bit_file.write(str(g_to_bin) + " ")
      elif (y % 2) == 0 and (x % 2) == 1:
        image_8bit_file.write(str(b_to_bin) + " ")
      elif (y % 2) == 1 and (x % 2) == 0:
        image_8bit_file.write(str(r_to_bin) + " ")
      elif (y % 2) == 1 and (x % 2) == 1:
        image_8bit_file.write(str(g_to_bin) + " ")

    image_rgb_file.write('\n')
    image_8bit_file.write('\n')

  image_rgb_file.close()
  image_8bit_file.close()
else:
  print("Error: Image must be of size 256x256")

rows:  256
columns:  256


### Script 2

Διαβάζουμε το txt και από 8-bit μετατρέπουμε τις τιμές σε int και κάνουμε Debayering ώστε να βρούμε τις τιμές που λείπουν, παράγουμε τις νέες RGB τιμές.

In [ ]:
import math

def write_line_to_file(file, pixels_line):
    for pixel in pixels_line:
      formatted_string =  str(pixel).replace(" ", "")
      file.write(formatted_string)
      file.write(" ")
    file.write('\n')

file_8bit = open(path + "image_8bit.txt", "r")
rgb_values_file = open(path + "script_rgb_values.txt", 'w')

data = []
for line in file_8bit:
  row = []
  values = line.split()

  for value in values:
    row.append(value)

  data.append(row)

file_8bit.close()
pixels = []
for row in range(1, 255):
  pixel_line = []
  starting_pixel = 0

  for column in range(1, 255):

    top_left = data[row-1][column-1]
    top_middle = data[row-1][column]
    top_right = data[row-1][column+1]

    middle_left = data[row][column-1]
    middle_middle = data[row][column]
    middle_right = data[row][column+1]

    bottom_left = data[row+1][column-1]
    bottom_middle = data[row+1][column]
    bottom_right = data[row+1][column+1]

    if (row % 2) == 0 and (column % 2) == 0:
      rgb = "G"
    elif (row % 2) == 0 and (column % 2) == 1:
      rgb = "B"
    elif (row % 2) == 1 and (column % 2) == 0:
      rgb = "R"
    elif (row % 2) == 1 and (column % 2) == 1:
      rgb = "G"

    r = 0
    g = 0
    b = 0

    if rgb == "R":
      r = int(middle_middle, 2)
      g = (int(top_middle, 2) + int(middle_left, 2) + int(middle_right, 2) + int(bottom_middle, 2)) / 4
      b = (int(top_left, 2) + int(top_right, 2) + int(bottom_left, 2) + int(bottom_right, 2)) / 4
    elif rgb == "G":
      r = (int(middle_left, 2) + int(middle_right, 2)) / 2
      g = int(middle_middle, 2)
      b = (int(top_middle, 2) + int(bottom_middle, 2)) / 2
    elif rgb == "B":
      r = (int(top_left, 2) + int(top_right, 2) + int(bottom_left, 2) + int(bottom_right, 2)) / 4
      g = (int(top_middle, 2) + int(middle_left, 2) + int(middle_right, 2) + int(bottom_middle, 2)) / 4
      b = int(middle_middle, 2)


    r = math.floor(r)
    g = math.floor(g)
    b = math.floor(b)

    pixel = (r,g,b)

    if (column == 1):
      starting_pixel = pixel

    pixel_line.append(pixel)

  pixel_line.insert(0, starting_pixel)
  pixel_line.append(pixel_line[254])

  pixels.append(pixel_line)

  if(row == 1):
    pixels.insert(0, pixel_line)
    write_line_to_file(rgb_values_file, pixel_line)
  if(row == 254):
    pixels.append(pixel_line)
    write_line_to_file(rgb_values_file, pixel_line)


  write_line_to_file(rgb_values_file, pixel_line)


array = np.array(pixels, dtype=np.uint8)

output_image = Image.fromarray(array)
output_image.save(path + "script_image.png")

rgb_values_file.close()


### Script 3

Οι RGB τιμές έχουν ήδη βρεθεί από κώδικα της C και πέρνουμε το παραγμένο RGB txt και φτιάχνουμε την εικόνα που προκύπτει

In [ ]:
output = open(path + "output.txt", "r")

output_data = []
for line in output:
  row = []
  values = line.split()

  for value in values:
    value = value.replace('(', '')
    value = value.replace(')', '')

    pixel = tuple(map(int, value.split(',')))
    row.append(pixel)

  output_data.append(row)

output.close()

array = np.array(output_data, dtype=np.uint8)

output_image = Image.fromarray(array)
output_image.save(path + "image_output.png")